In [1]:
import pandas as pd
import string 
import re #regex library


from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


data = pd.read_excel("reviews.xlsx")
selected_columns = ['Label', 'Reviews']
selected_data = data[selected_columns]
data = selected_data
data

,Label,Reviews
0,1,kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...
1,0,"Jahitannya sih rapi,cuman ada benang yang ikut..."
2,0,Sesuai harga. Agak tipis tapi masih oke kok. W...
3,1,"Wah gila sihhh sebagus itu, se worth it, se l..."
4,0,Kain nya bagus halus \nTapi kok di bukak koto...
...,...,...
826,1,Terima kasih barang sudah sampai sesuai ukuran...
827,1,Mantapp realpicttt bangttt tapi pengemasan nya...
828,1,"Suka bgt sama tasnya, ga kayak tas local. Kere..."
829,1,kualitas produk sangat baik. produk original. ...


In [2]:
data["Reviews"] = data["Reviews"].str.lower()
data

,Label,Reviews
0,1,kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...
1,0,"jahitannya sih rapi,cuman ada benang yang ikut..."
2,0,sesuai harga. agak tipis tapi masih oke kok. w...
3,1,"wah gila sihhh sebagus itu, se worth it, se l..."
4,0,kain nya bagus halus \ntapi kok di bukak koto...
...,...,...
826,1,terima kasih barang sudah sampai sesuai ukuran...
827,1,mantapp realpicttt bangttt tapi pengemasan nya...
828,1,"suka bgt sama tasnya, ga kayak tas local. kere..."
829,1,kualitas produk sangat baik. produk original. ...


# Tokenizing, dan Cleansing

In [3]:

def remove(text):
    text = re.sub(r'[0-9]', ' ', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'[^A-Za-z\s]', '', text)
    text = re.sub(r'(\w)\1+', r'\1', text)
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('\n', '')
    text = text.replace('_', '')
    return text

data['Reviews'] = data['Reviews'].apply(remove)

# NLTK word rokenize 
def tokenize(text):
    return text.split()

data['Reviews_tokens'] = data['Reviews'].apply(tokenize)
data

,Label,Reviews,Reviews_tokens
0,1,kemeja nya bagus bgt a mau ngis knpa ga dri dl...,"[kemeja, nya, bagus, bgt, a, mau, ngis, knpa, ..."
1,0,jahitanya sih rapi cuman ada benang yang ikut ...,"[jahitanya, sih, rapi, cuman, ada, benang, yan..."
2,0,sesuai harga agak tipis tapi masih oke kok war...,"[sesuai, harga, agak, tipis, tapi, masih, oke,..."
3,1,wah gila sih sebagus itu se worth it se lembut...,"[wah, gila, sih, sebagus, itu, se, worth, it, ..."
4,0,kain nya bagus halus tapi kok di bukak kotor y...,"[kain, nya, bagus, halus, tapi, kok, di, bukak..."
...,...,...,...
826,1,terima kasih barang sudah sampai sesuai ukuran...,"[terima, kasih, barang, sudah, sampai, sesuai,..."
827,1,mantap realpict bangt tapi pengemasan nya cuma...,"[mantap, realpict, bangt, tapi, pengemasan, ny..."
828,1,suka bgt sama tasnya ga kayak tas local keren ...,"[suka, bgt, sama, tasnya, ga, kayak, tas, loca..."
829,1,kualitas produk sangat baik produk original ha...,"[kualitas, produk, sangat, baik, produk, origi..."


### Stopwords removal

In [4]:

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "nya", "sih"])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data['Reviews_Stopword'] = data['Reviews_tokens'].apply(stopwords_removal) 
data

,Label,Reviews,Reviews_tokens,Reviews_Stopword
0,1,kemeja nya bagus bgt a mau ngis knpa ga dri dl...,"[kemeja, nya, bagus, bgt, a, mau, ngis, knpa, ...","[kemeja, bagus, bgt, a, ngis, knpa, ga, dri, d..."
1,0,jahitanya sih rapi cuman ada benang yang ikut ...,"[jahitanya, sih, rapi, cuman, ada, benang, yan...","[jahitanya, rapi, cuman, benang, jahit, jelek]"
2,0,sesuai harga agak tipis tapi masih oke kok war...,"[sesuai, harga, agak, tipis, tapi, masih, oke,...","[sesuai, harga, tipis, oke, warnanya, abu, kal..."
3,1,wah gila sih sebagus itu se worth it se lembut...,"[wah, gila, sih, sebagus, itu, se, worth, it, ...","[gila, sebagus, worth, it, lembut, bajunya, ki..."
4,0,kain nya bagus halus tapi kok di bukak kotor y...,"[kain, nya, bagus, halus, tapi, kok, di, bukak...","[kain, bagus, halus, bukak, kotor, ya, warna, ..."
...,...,...,...,...
826,1,terima kasih barang sudah sampai sesuai ukuran...,"[terima, kasih, barang, sudah, sampai, sesuai,...","[terima, kasih, barang, sesuai, ukuran, sesuai..."
827,1,mantap realpict bangt tapi pengemasan nya cuma...,"[mantap, realpict, bangt, tapi, pengemasan, ny...","[mantap, realpict, bangt, pengemasan, cuman, p..."
828,1,suka bgt sama tasnya ga kayak tas local keren ...,"[suka, bgt, sama, tasnya, ga, kayak, tas, loca...","[suka, bgt, tasnya, ga, kayak, tas, local, ker..."
829,1,kualitas produk sangat baik produk original ha...,"[kualitas, produk, sangat, baik, produk, origi...","[kualitas, produk, produk, original, harga, pr..."


# Normalisasi

In [5]:

normalizad_word = pd.read_excel("normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

data['reviews_normalized'] = data['Reviews_Stopword'].apply(normalized_term)

data['reviews_normalized'].head(10)

0    [kemeja, bagus, banget, a, ngis, knpa, tidak, ...
1       [jahitanya, rapi, cuman, benang, jahit, jelek]
2    [sesuai, harga, tipis, oke, warnanya, abu, kal...
3    [gila, sebagus, worth, it, lembut, bajunya, ki...
4    [kain, bagus, halus, buka, kotor, ya, warna, p...
5    [bagus, deh, harga, segitu, kainya, tidak, tip...
6    [harga, mahal, bahan, tipis, banget, pengirima...
7    [puas, banget, takut, jelek, foto, yearbok, pa...
8    [kecewa, karena, pesan, warna, fuschia, dateng...
9    [maf, beri, bintang, tidak, sesuai, gambar, di...
Name: reviews_normalized, dtype: object

In [6]:

# stemmed
def stemming(token):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return [stemmer.stem(word) for word in token]

data['stemming'] = data['reviews_normalized'].apply(lambda x: stemming(x))
data

,Label,Reviews,Reviews_tokens,Reviews_Stopword,reviews_normalized,stemming
0,1,kemeja nya bagus bgt a mau ngis knpa ga dri dl...,"[kemeja, nya, bagus, bgt, a, mau, ngis, knpa, ...","[kemeja, bagus, bgt, a, ngis, knpa, ga, dri, d...","[kemeja, bagus, banget, a, ngis, knpa, tidak, ...","[kemeja, bagus, banget, a, ngis, knpa, tidak, ..."
1,0,jahitanya sih rapi cuman ada benang yang ikut ...,"[jahitanya, sih, rapi, cuman, ada, benang, yan...","[jahitanya, rapi, cuman, benang, jahit, jelek]","[jahitanya, rapi, cuman, benang, jahit, jelek]","[jahitanya, rapi, cuman, benang, jahit, jelek]"
2,0,sesuai harga agak tipis tapi masih oke kok war...,"[sesuai, harga, agak, tipis, tapi, masih, oke,...","[sesuai, harga, tipis, oke, warnanya, abu, kal...","[sesuai, harga, tipis, oke, warnanya, abu, kal...","[sesuai, harga, tipis, oke, warna, abu, kalau,..."
3,1,wah gila sih sebagus itu se worth it se lembut...,"[wah, gila, sih, sebagus, itu, se, worth, it, ...","[gila, sebagus, worth, it, lembut, bajunya, ki...","[gila, sebagus, worth, it, lembut, bajunya, ki...","[gila, bagus, worth, it, lembut, baju, kirain,..."
4,0,kain nya bagus halus tapi kok di bukak kotor y...,"[kain, nya, bagus, halus, tapi, kok, di, bukak...","[kain, bagus, halus, bukak, kotor, ya, warna, ...","[kain, bagus, halus, buka, kotor, ya, warna, p...","[kain, bagus, halus, buka, kotor, ya, warna, p..."
...,...,...,...,...,...,...
826,1,terima kasih barang sudah sampai sesuai ukuran...,"[terima, kasih, barang, sudah, sampai, sesuai,...","[terima, kasih, barang, sesuai, ukuran, sesuai...","[terima, kasih, barang, sesuai, ukuran, sesuai...","[terima, kasih, barang, sesuai, ukur, sesuai, ..."
827,1,mantap realpict bangt tapi pengemasan nya cuma...,"[mantap, realpict, bangt, tapi, pengemasan, ny...","[mantap, realpict, bangt, pengemasan, cuman, p...","[mantap, realpict, bangt, pengemasan, cuman, p...","[mantap, realpict, bangt, emas, cuman, plastik..."
828,1,suka bgt sama tasnya ga kayak tas local keren ...,"[suka, bgt, sama, tasnya, ga, kayak, tas, loca...","[suka, bgt, tasnya, ga, kayak, tas, local, ker...","[suka, banget, tasnya, tidak, seperti, tas, lo...","[suka, banget, tas, tidak, seperti, tas, local..."
829,1,kualitas produk sangat baik produk original ha...,"[kualitas, produk, sangat, baik, produk, origi...","[kualitas, produk, produk, original, harga, pr...","[kualitas, produk, produk, original, harga, pr...","[kualitas, produk, produk, original, harga, pr..."


In [7]:
data.to_excel("Hasil_Preprocessing.xlsx")